# Laboratorio 3 | Análisis GeoEspacial y Sensores Remotos

Diego Duarte 

Paula Barillas


## Parte 1: Acceso y Búsqueda
1. Registrarse e iniciar sesión en [Copernicus Data Space](https://dataspace.copernicus.eu/).
2. Buscar imágenes Sentinel-2 L2A:
   - **Área:** Región del Petén
   - **Fechas:** Enero 2020 y Enero 2024
   - **Cobertura de nubes:** ≤ 10%

## Parte 2: Descarga y Preprocesamiento
1. Recortar el área de interés usando una caja o shapefile.
2. Aplicar enmascarado de nubes si es posible (QA60 o SCL).
3. Descargar las imágenes bandas 4 (Rojo) y 8 (Infrarrojo Cercano) en formato .tiff.
4. Usar rasterio para cargar las imágenes al Jupyter Notebook.

## Parte 3: Cálculo de NDVI y Detección de Cambios
1. Calcular NDVI:
   - NDVI = (NIR − Rojo) / (NIR + Rojo)
2. Crear imagen de diferencia: NDVI_2024 - NDVI_2020
3. Aplicar un umbral (ej. < -0.2) para marcar pérdida significativa de vegetación.

## Parte 4: Visualización y Reporte
1. Graficar:
   - Mapas NDVI de 2020 y 2024
   - Imagen de diferencia
   - Máscara de deforestación
2. Calcular el área (%) de deforestación.


In [ ]:
# Parte 3: Cálculo de NDVI y Detección de Cambios
import rasterio
import numpy as np
import matplotlib.pyplot as plt

# Cargar bandas 
red_2020 = rasterio.open('B4_2020.tif').read(1).astype('float32')
nir_2020 = rasterio.open('B8_2020.tif').read(1).astype('float32')
red_2024 = rasterio.open('B4_2024.tif').read(1).astype('float32')
nir_2024 = rasterio.open('B8_2024.tif').read(1).astype('float32')

# Calcular NDVI
ndvi_2020 = (nir_2020 - red_2020) / (nir_2020 + red_2020)
ndvi_2024 = (nir_2024 - red_2024) / (nir_2024 + red_2024)

# Imagen de diferencia
ndvi_diff = ndvi_2024 - ndvi_2020

# Máscara de deforestación (umbral < -0.2)
def_mask = ndvi_diff < -0.2

# Calcular área (%) de deforestación
area_def = np.sum(def_mask) / def_mask.size * 100
print(f"Área de deforestación: {area_def:.2f}%")

# Parte 4: Visualización
fig, axs = plt.subplots(1, 4, figsize=(20, 5))
axs[0].imshow(ndvi_2020, cmap='RdYlGn')
axs[0].set_title('NDVI 2020')
axs[1].imshow(ndvi_2024, cmap='RdYlGn')
axs[1].set_title('NDVI 2024')
axs[2].imshow(ndvi_diff, cmap='bwr', vmin=-1, vmax=1)
axs[2].set_title('Diferencia NDVI')
axs[3].imshow(def_mask, cmap='gray')
axs[3].set_title('Máscara de Deforestación')
for ax in axs:
    ax.axis('off')
plt.tight_layout()
plt.show()